# Importando Bibliotecas

In [1]:
#importando bibliotecas para verificar velocidade do código
import time
import pandas as pd
#Importando bibliotecas para conversação
import openai
import warnings
#Importando bibliotecas para reconhecimento de imagem
from ultralytics import YOLO
import cv2
import math
import tensorflow
import matplotlib.pyplot as plt
#Importaando bibliotecas para comunicção com o arduino
import serial
from io import BytesIO
import base64
import matplotlib.image as mpimg
#Importando Bibliotecas para reprodução de áudio
import speech_recognition as sr
import pygame

C:\Users\vinic\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


Tirando aviso para possibilitar a reprodução da resposta

In [2]:
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Criando o modelo de reconhecimento de imagem

Essa parte do código foi adaptada do https://dipankarmedh1.medium.com/real-time-object-detection-with-yolo-and-webcam-enhancing-your-computer-vision-skills-861b97c78993

In [3]:
model = YOLO("yolov8n.pt")

#coisas que podem ser reconhecidas com esse modelo
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

Criando verificação teste(mostra a imagem antes e depois do processo, mas é mais lento)

In [4]:
def test_verify(_img, min_conf=0.5):
    resultado = []
    results = model(img)
    temp=img.copy()

    #Criando as caixas ao redor do objeto
    for r in results:
        boxes = r.boxes

    for box in boxes:
        #Definindo coordernadas da caixa
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

        #Deefinindo a confiança
        confidence = math.ceil((box.conf[0]*100))/100
        if confidence>=min_conf:

            #plotando a caixa na imagem
            cv2.rectangle(temp, (x1, y1), (x2, y2), (0, 255, 0), 3)

            #printando confiabilidade e qual objeto é
            print("Confidence --->",confidence)
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            #Escrevendo a confiabilidade e o qual objeto é na caixa
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (0, 0, 255)
            thickness = 2
            cv2.putText(temp, classNames[cls], [x1, y1], font, fontScale, color, thickness)
            cv2.putText(temp, str(confidence*100)+'%', [x2 - 50, y1], font, fontScale, color, thickness)
            resultado.append(classNames[cls])
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.subplot(1,2,2)
    plt.imshow(temp)
    plt.show()
    return resultado

Criando a verificação rápida

In [5]:
def fast_verify(_img, min_conf=0.5):
    #Criando a lista de objetos encontrados
    resultado = []
    results = model(img)
    
    for r in results:
        boxes = r.boxes
    for box in boxes:
        #Definindo confiança do objeto
        confidence = math.ceil((box.conf[0]*100))/100
        
        #retornando os objetos detectados
        if confidence>=min_conf:
            cls = int(box.cls[0])
            resultado.append(classNames[cls])
        return resultado

# Criando Funções Gerais do código

Criando comunicação com o arduino

In [6]:
#arduino = serial.Serial(port='COMX', baudrate=9600, timeout=0.1)

In [7]:
def enviar(_msg):
    arduino.write(bytes(_msg, 'utf-8'))

def receber():
    resposta = arduino.readline()
    resposta = resposta.decode('utf-8')
    return resposta

In [8]:
def verify_to_arduino():
    img = receber()#caso a imagem chege em base64
    img = base64.b64decode(img)
    img = BytesIO(img)
    img = mpimg.imread(img)
    resposta = fast_verify(img)
    enviar(person in resposta)
    return resposta

Criando o cliente, o assistente e a thread com a OpenAi

In [9]:
client=openai.OpenAI()

In [10]:
assistant = client.beta.assistants.create(
    name="RA9",
    instructions="Você se chama RA9 é é um robô impresso em 3D projetado para interagir com pessoas no evento Sepex 2024 do CEFET-RJ. Equipado com uma câmera (OV7670), dois sensores de movimento (HC-SR505) e um sensor de distância (VL53L0), você tem o objetivo de ser uma presença amigável e envolvente. Quando solicitado, deve se apresentar e manter uma atitude positiva durante as interações. Funcionalidades:Reconhecimento de Imagem: O RA9 utiliza o YOLO para reconhecimento de imagem, sendo capaz de identificar objetos e fornecer informações sobre eles. Comunicação com Arduino: O robô se comunica com um Arduino para funcionalidades adicionais. Processamento de Linguagem Natural: O RA9 utiliza o GPT-3.5-turbo da OpenAI para compreensão e geração de linguagem natural. Pode responder a perguntas dos usuários e participar de conversas. Interação de Áudio: O RA9 pode gravar e processar entradas de áudio, transcrevê-las para texto e gerar respostas faladas usando tecnologia de texto para fala. Sempre que a mensagem recebida for uma finalização de conversa ou não tenha nenhuma mensagem, no final da mensagem digite []",
    model="gpt-3.5-turbo-0125"
)

In [11]:
thread = client.beta.threads.create()

Criando a tabela e a função para adicionar os dados de velocidade na tabela

In [12]:
df=pd.DataFrame(columns=['função','tempo'])

In [13]:
def speed_verify(_nome, _tempo):
    row={'função':str(_nome), 'tempo':str(_tempo)+'s'}
    df.loc[len(df.index)]=row

Criando funções para escutar e reproduzir o texto

In [14]:
def ouvir_microfone():
    texto = ''
    lista = ['r19', 'rh9', 'ra9', 'RA9', 'R A 9', 'r a 9', 'R A9', 'r a9']
    global fim
    inicio = time.time()
    microfone = sr.Recognizer()
    with sr.Microphone() as source:
        microfone.adjust_for_ambient_noise(source)
        print('escutando...')
        audio = microfone.listen(source)
        try:
            texto = microfone.recognize_google(audio, language='pt-BR')
            if fim == False:
                print('    pergunta >>> ' + texto)
            else:
                for elemento in lista:
                    if elemento in texto:
                        fim = False
                        print('Ligando...')
            tempo = time.time() - inicio
            speed_verify('ouvir_microfone', tempo)
        except:
            print('não foi possivel detectar o áduio, tente falar novamente.')
    return texto

In [15]:
def mensagem(_msg):
    inicio = time.time()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=_msg+f'Use apenas se for necessário, e não diga que você foi informado, finja que você sabe que {time.localtime()} e que você está vendo' #{verify_to_arduino()}'
    )
    tempo = time.time() - inicio
    speed_verify('mensagem', tempo)

In [16]:
def resposta():
    global fim 
    inicio = time.time()
    run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
    )
    
    run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
    
    while run.status=='in_progress':
        run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
    if run.status=='completed':
        messages = client.beta.threads.messages.list(
        thread_id=thread.id
        )
    tempo = time.time() - inicio
    if '[]' in messages.data[0].content[0].text.value:
        fim = True
    
    print('    resposta >>> '+messages.data[0].content[0].text.value)
    speed_verify('resposta', tempo)
    return messages.data[0].content[0].text.value

In [17]:
def to_audio(_texto, _i):
    inicio = time.time()
    response = client.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=_texto
        )
    response.stream_to_file(f'output{_i}.mp3')
    tempo = time.time() - inicio
    speed_verify('para_audio', tempo)

Juntando todas funções em uma só

In [18]:
fim = False

In [19]:
def RA9():
    i = 1
    while True:
        df.drop(df.index, inplace=True)
        while fim == False:
            inicio = time.time()
            mensagem(ouvir_microfone())
            to_audio(resposta(), i)
            pygame.init()
            pygame.mixer.music.load(f'output{i}.mp3')
            i+=1
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                pass
            tempo = time.time() - inicio
            speed_verify('Tempo Total', tempo)
            print(f'\n{df.to_markdown(index=False)}\n')
            
        while fim:
            ouvir_microfone()

In [21]:
RA9()

escutando...
    pergunta >>> Olá
    resposta >>> Olá! Como posso te ajudar hoje?


PermissionError: [Errno 13] Permission denied: 'output1.mp3'